**LC dataset: Processes the BUG files into files prepared for use in R**

This notebook processes the output from the fastq file processing for this dataset. The data produced here is pre-generated and downloaded by the figure generation code. The purpose of this processing step is to prepare the data for figure generation, by filtering the data and producing downsampled datasets in addition to the original one. 

Steps:
1. Clone the code repo and download data to process
2. Prepare the R environment
3. Process the data
4. Generate statistics for the dataset

**1. Clone the code repo and download data to process**

In [1]:
![ -d "GRNP_2020" ] && rm -r GRNP_2020

!git clone https://github.com/pachterlab/GRNP_2020.git


Cloning into 'GRNP_2020'...
remote: Enumerating objects: 365, done.
remote: Counting objects: 100% (365/365), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 788 (delta 232), reused 104 (delta 55), pack-reused 423
Receiving objects: 100% (788/788), 7.43 MiB | 17.46 MiB/s, done.
Resolving deltas: 100% (460/460), done.


In [2]:
#download BUG data from Zenodo
!mkdir data
!cd data && wget https://zenodo.org/record/3924675/files/LC.zip?download=1 && unzip 'LC.zip?download=1' && rm 'LC.zip?download=1'

--2020-07-02 06:36:16--  https://zenodo.org/record/3924675/files/LC.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 782942049 (747M) [application/octet-stream]
Saving to: ‘LC.zip?download=1’

LC.zip?download=1   100%[===================>] 746.67M  3.70MB/s    in 1m 55s  

2020-07-02 06:38:13 (6.49 MB/s) - ‘LC.zip?download=1’ saved [782942049/782942049]

Archive:  LC.zip?download=1
   creating: LC/
   creating: LC/bus_output/
  inflating: LC/bus_output/bug.txt   
  inflating: LC/bus_output/coll.genes.txt  
  inflating: LC/bus_output/transcripts_to_genes.txt  


In [3]:
#Check that download worked
!cd data && ls -l && cd LC/bus_output && ls -l

total 4
drwxr-xr-x 3 root root 4096 Jul  1 00:00 LC
total 4043628
-rw-r--r-- 1 root root 4131588912 Jun 30 05:04 bug.txt
-rw-r--r-- 1 root root     738211 Jun 30 01:24 coll.genes.txt
-rw-r--r-- 1 root root    8335642 Jun 30 01:00 transcripts_to_genes.txt


**2. Prepare the R environment**

In [4]:
#switch to R mode
%reload_ext rpy2.ipython


/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [5]:
#install the R packages
%%R
sourcePath = "GRNP_2020/NotebookAdaptedRCode/"
install.packages("qdapTools")
install.packages("dplyr")
install.packages("stringdist")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘bitops’, ‘chron’, ‘data.table’, ‘RCurl’, ‘XML’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/bitops_1.0-6.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 8734 bytes

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

**3. Process the data**

Here we discard multimapped UMIs and all UMIs belonging to cells with fewer than 200 UMIs. We also precalculate gene expression, fraction of single-copy molecules etc. and save as stats (statistics). These can later be used when generating figures. We also generate down-sampled BUGs.

In [6]:
#create output directory
!mkdir figureData

In [7]:
#First set some path variables
%%R
source("GRNP_2020/RCode/pathsGoogleColab.R")


In [ ]:
#Process and filter the BUG file
%%R
source(paste0(sourcePath, "BUGProcessingHelpers.R"))
createStandardBugsData(paste0(dataPath,"LC/"), "LC", c(0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1), UmisPerCellLimit = 1000)



R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘qdapTools’


R[write to console]: The following object is masked from ‘package:dplyr’:

    id




[1] "Generating data for LC"
[1] "Reading BUG from data/LC/ ..."
[1] "Filtering multi-mapped reads..."


**4. Generate statistics for the dataset**

Here we create a file with various statistics for the dataset, which is used for generating table S2. It also contains some additional information about the dataset. Generation of this file may take several hours.

In [ ]:
%%R
source(paste0(sourcePath, "GenBugSummary.R"))
genBugSummary("LC", "FGF23", "RPS10", 10)

In [ ]:
!cd figureData/LC && ls -l && cat ds_summary.txt